In [1]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn

In [12]:
import pandas
import nltk.stem
from sklearn import tree
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold

# 簡單範例

In [48]:
string = "happy"

synset_list = wn.synsets(string)

In [49]:
synset_list

[Synset('happy.a.01'),
 Synset('felicitous.s.02'),
 Synset('glad.s.02'),
 Synset('happy.s.04')]

In [50]:
synset_list[0].name()

'happy.a.01'

In [51]:
senti_score = swn.senti_synset(synset_list[0].name())
print(senti_score)

<happy.a.01: PosScore=0.875 NegScore=0.0>


In [52]:
senti_score.neg_score()

0.0

In [53]:
senti_score.pos_score()

0.875

In [54]:
senti_score.obj_score()

0.125

In [55]:
senti_score.synset.name()

'happy.a.01'

In [56]:
for ii in synset_list:
    print("{0}, Positive:{1}, Negative:{2}".format(swn.senti_synset(ii.name()).synset.name(),
                                                   swn.senti_synset(ii.name()).pos_score(),
                                                   swn.senti_synset(ii.name()).neg_score()))

happy.a.01, Positive:0.875, Negative:0.0
felicitous.s.02, Positive:0.75, Negative:0.0
glad.s.02, Positive:0.5, Negative:0.0
happy.s.04, Positive:0.125, Negative:0.0


# 甚麼是詞幹Stem?

In [26]:
s = nltk.stem.PorterStemmer()

In [27]:
s.stem("image")

'imag'

In [28]:
s.stem("imaging")

'imag'

In [29]:
s.stem("imagine")

'imagin'

In [30]:
s.stem("imagination")

'imagin'

# 讀檔

In [3]:
seed = 1234
dataframe = pandas.read_csv("imdb_labelled.csv", header=None, encoding="big5")
dataset = dataframe.values
X = dataset[:,0].astype(str)
Y = dataset[:,1].astype(int)

In [4]:
X

array([ 'A very, very, very slow-moving, aimless movie about a distressed, drifting young man.  ',
       'Not sure who was more lost - the flat characters or the audience, nearly half of whom walked out.  ',
       'Attempting artiness with black & white and clever camera angles, the movie disappointed - became even more ridiculous - as the acting was poor and the plot and lines almost non-existent.  ',
       'Very little music or anything to speak of.  ',
       'The best scene in the movie was when Gerardo is trying to find a song that keeps running through his head.  ',
       "The rest of the movie lacks art, charm, meaning... If it's about emptiness, it works I guess because it's empty.  ",
       'Wasted two hours.  ',
       'Saw the movie today and thought it was a good effort, good messages for kids.  ',
       'A bit predictable.  ',
       'Loved the casting of Jimmy Buffet as the science teacher.  ',
       'And those baby owls were adorable.  ',
       "The movie showed 

# 擴充CountVectorizer具有Stemming功能

In [5]:
import nltk.stem
english_stemmer = nltk.stem.SnowballStemmer('english')

# 繼承用法--> MyStemmedCountVectorizer 繼承 CountVectorizer  並進行 函數 取代 override
class MyStemmedCountVectorizer( CountVectorizer ): 
    def build_analyzer(self):
        analyzer = super(MyStemmedCountVectorizer, self).build_analyzer()        
        return lambda doc: (english_stemmer.stem(w) for w in analyzer(doc))

In [6]:
vect = MyStemmedCountVectorizer(min_df=1, stop_words='english')

In [60]:
type(X)

numpy.ndarray

In [7]:
Xvec=vect.fit_transform(X)

In [ ]:
#pandas.DataFrame()

In [8]:
print(vect.get_feature_names())

['10', '10月10日', '12', '13', '15', '15pm', '17', '18th', '1928', '1947', '1948', '1949', '1971', '1973', '1980', '1986', '1995', '1998', '20', '2005', '2006', '20th', '25', '30', '40', '50', '54', '70', '70000', '70s', '80', '80s', '8pm', '90', '95', 'aailiyah', 'abandon', 'abil', 'abroad', 'absolut', 'abstrus', 'abysm', 'academi', 'accent', 'access', 'acclaim', 'accolad', 'accur', 'accus', 'achiev', 'achill', 'ackerman', 'act', 'action', 'actor', 'actress', 'actual', 'ad', 'adam', 'adapt', 'add', 'addit', 'admin', 'admir', 'admit', 'ador', 'adrift', 'adventur', 'advis', 'aerial', 'aesthet', 'affect', 'affleck', 'afraid', 'africa', 'afternoon', 'age', 'ago', 'agre', 'aimless', 'air', 'akasha', 'akin', 'alert', 'alexand', 'alik', 'allison', 'allow', 'alongsid', 'amateurish', 'amaz', 'america', 'american', 'amus', 'anatomist', 'angel', 'angela', 'angelina', 'angl', 'angri', 'anguish', 'angus', 'anim', 'anita', 'ann', 'anniversari', 'annoy', 'anthoni', 'antithesi', 'apart', 'appal', 'appe

In [9]:
type(vect.get_feature_names())

list

# 把每個詞放入Synsets這個方法，找出所有單子的詞性

In [15]:
sentiment_score_list = []
for senti_word_text in vect.get_feature_names():
    synsets_adj = wn.synsets(senti_word_text,pos=wn.ADJ)
    synsets_adv = wn.synsets(senti_word_text,pos=wn.ADV)
    synsets_verb = wn.synsets(senti_word_text,pos=wn.VERB)
    synsets_noun = wn.synsets(senti_word_text,pos=wn.NOUN)
    temp = synsets_adj+synsets_adv+synsets_verb+synsets_noun
    sentiment_score_list = sentiment_score_list + temp
print(sentiment_score_list) 

[Synset('ten.s.01'), Synset('ten.n.01'), Synset('twelve.s.01'), Synset('twelve.n.01'), Synset('thirteen.s.01'), Synset('thirteen.n.01'), Synset('fifteen.s.01'), Synset('fifteen.n.01'), Synset('seventeen.s.01'), Synset('seventeen.n.01'), Synset('eighteenth.s.01'), Synset('twenty.s.01'), Synset('twenty.n.01'), Synset('twentieth.s.01'), Synset('twenty-five.s.01'), Synset('twenty-five.n.01'), Synset('thirty.s.01'), Synset('thirty.n.01'), Synset('forty.s.01'), Synset('forty.n.01'), Synset('fifty.s.01'), Synset('fifty.n.01'), Synset('fifty-four.s.01'), Synset('seventy.s.01'), Synset('seventy.n.01'), Synset('seventy.n.01'), Synset('eighty.s.01'), Synset('eighty.n.01'), Synset('eighty.n.01'), Synset('ninety.s.01'), Synset('ninety.n.01'), Synset('ninety-five.s.01'), Synset('abandon.v.01'), Synset('abandon.v.02'), Synset('vacate.v.02'), Synset('abandon.v.04'), Synset('abandon.v.05'), Synset('abandon.n.01'), Synset('wildness.n.01'), Synset('abroad.s.01'), Synset('abroad.r.01'), Synset('afield.r.0

# 計算每個字的情緒分數

In [16]:
total = 0
total2 = 0
for senti_word_text in sentiment_score_list:
    senti_synset = swn.senti_synset(senti_word_text.name())
    print("| {0}的正向情緒分數:{1} | 負向情緒分數:{2} | 客觀情緒分數:{3} |".format(senti_synset.synset.name(),
                                    senti_synset.pos_score(),
                                   senti_synset.neg_score(),
                                   senti_synset.obj_score()))
    total2 = senti_synset.neg_score()+total2
    total = senti_synset.pos_score()+total
    

| ten.s.01的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| ten.n.01的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| twelve.s.01的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| twelve.n.01的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| thirteen.s.01的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| thirteen.n.01的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| fifteen.s.01的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| fifteen.n.01的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| seventeen.s.01的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| seventeen.n.01的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| eighteenth.s.01的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| twenty.s.01的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| twenty.n.01的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| twentieth.s.01的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| twenty-five.s.01的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| twenty-five.n.01的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| thirty.s.01的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| thirty.n.01的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| forty.s.01的正向情緒分數:0.0

# 判斷整個文件的情緒狀態

In [17]:
print("正向情緒分數加總",total)
print("負向情緒分數加總",total2)
if(total>total2):
    print("正面評論")
elif(total<total2):
    print("負面評論")

正向情緒分數加總 671.6110000000001
負向情緒分數加總 665.264
正面評論


# 只判斷形容詞和副詞

In [32]:
sentiment_score_list = []
for senti_word_text in vect.get_feature_names():
    synsets_adj = wn.synsets(senti_word_text,pos=wn.ADJ)
    synsets_adv = wn.synsets(senti_word_text,pos=wn.ADV)
    #synsets_verb = wn.synsets(senti_word_text,pos=wn.VERB)
    #synsets_noun = wn.synsets(senti_word_text,pos=wn.NOUN)
    temp = synsets_adj+synsets_adv
    sentiment_score_list = sentiment_score_list + temp
print(sentiment_score_list) 

[Synset('ten.s.01'), Synset('twelve.s.01'), Synset('thirteen.s.01'), Synset('fifteen.s.01'), Synset('seventeen.s.01'), Synset('eighteenth.s.01'), Synset('twenty.s.01'), Synset('twentieth.s.01'), Synset('twenty-five.s.01'), Synset('thirty.s.01'), Synset('forty.s.01'), Synset('fifty.s.01'), Synset('fifty-four.s.01'), Synset('seventy.s.01'), Synset('eighty.s.01'), Synset('ninety.s.01'), Synset('ninety-five.s.01'), Synset('abroad.s.01'), Synset('abroad.r.01'), Synset('afield.r.01'), Synset('overseas.r.02'), Synset('actual.a.01'), Synset('actual.s.02'), Synset('actual.s.03'), Synset('actual.s.04'), Synset('actual.s.05'), Synset('ad.r.01'), Synset('adrift.s.01'), Synset('adrift.s.02'), Synset('adrift.r.01'), Synset('adrift.r.02'), Synset('aerial.a.01'), Synset('aeriform.s.02'), Synset('afraid.a.01'), Synset('afraid.s.02'), Synset('afraid.s.03'), Synset('afraid.s.04'), Synset('ago.s.01'), Synset('ago.r.01'), Synset('adrift.s.01'), Synset('aimless.s.02'), Synset('akin.s.01'), Synset('akin.s.02

In [33]:
total = 0
total2 = 0
for senti_word_text in sentiment_score_list:
    senti_synset = swn.senti_synset(senti_word_text.name())
    print("| {0}的正向情緒分數:{1} | 負向情緒分數:{2} | 客觀情緒分數:{3} |".format(senti_synset.synset.name(),
                                    senti_synset.pos_score(),
                                   senti_synset.neg_score(),
                                   senti_synset.obj_score()))
    total2 = senti_synset.neg_score()+total2
    total = senti_synset.pos_score()+total

| ten.s.01的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| twelve.s.01的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| thirteen.s.01的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| fifteen.s.01的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| seventeen.s.01的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| eighteenth.s.01的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| twenty.s.01的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| twentieth.s.01的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| twenty-five.s.01的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| thirty.s.01的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| forty.s.01的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| fifty.s.01的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| fifty-four.s.01的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| seventy.s.01的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| eighty.s.01的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| ninety.s.01的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| ninety-five.s.01的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| abroad.s.01的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| abroad.r.01的正向情緒分數:0.

In [34]:
print("正向情緒分數加總",total)
print("負向情緒分數加總",total2)
if(total>total2):
    print("正面評論")
elif(total<total2):
    print("負面評論")

正向情緒分數加總 330.583
負向情緒分數加總 334.667
負面評論


In [31]:
#print(','.join(map(str, vect.get_feature_names())))

# 範例評論

In [18]:
sent="These are the central themes of the film and they are handled ineptly, stereotypically and with no depth of imagination."  

In [ ]:
這些是電影所要表達的主題，它們是無理的，刻板印象的，沒有想像力的深度

# 段詞(使用簡單的方法)

In [20]:
try_sentence = "These are the central themes of the film and they are handled ineptly, stereotypically and with no depth of imagination."
try_sentence = try_sentence.split()
print(try_sentence)

['These', 'are', 'the', 'central', 'themes', 'of', 'the', 'film', 'and', 'they', 'are', 'handled', 'ineptly,', 'stereotypically', 'and', 'with', 'no', 'depth', 'of', 'imagination.']


In [37]:
type(try_sentence)

list

In [38]:
sentiment_score_list = []
for senti_word_text in try_sentence:
    synsets_adj = wn.synsets(senti_word_text,pos=wn.ADJ)
    synsets_adv = wn.synsets(senti_word_text,pos=wn.ADV)
    synsets_verb = wn.synsets(senti_word_text,pos=wn.VERB)
    synsets_noun = wn.synsets(senti_word_text,pos=wn.NOUN)
    temp = synsets_adj+synsets_adv+synsets_verb+synsets_noun
    sentiment_score_list = sentiment_score_list + temp
print(sentiment_score_list) 

[Synset('be.v.01'), Synset('be.v.02'), Synset('be.v.03'), Synset('exist.v.01'), Synset('be.v.05'), Synset('equal.v.01'), Synset('constitute.v.01'), Synset('be.v.08'), Synset('embody.v.02'), Synset('be.v.10'), Synset('be.v.11'), Synset('be.v.12'), Synset('cost.v.01'), Synset('are.n.01'), Synset('cardinal.s.01'), Synset('central.a.02'), Synset('central.n.01'), Synset('theme.v.01'), Synset('subject.n.01'), Synset('theme.n.02'), Synset('theme.n.03'), Synset('composition.n.08'), Synset('root.n.03'), Synset('film.v.01'), Synset('film.v.02'), Synset('movie.n.01'), Synset('film.n.02'), Synset('film.n.03'), Synset('film.n.04'), Synset('film.n.05'), Synset('be.v.01'), Synset('be.v.02'), Synset('be.v.03'), Synset('exist.v.01'), Synset('be.v.05'), Synset('equal.v.01'), Synset('constitute.v.01'), Synset('be.v.08'), Synset('embody.v.02'), Synset('be.v.10'), Synset('be.v.11'), Synset('be.v.12'), Synset('cost.v.01'), Synset('are.n.01'), Synset('handled.a.01'), Synset('manage.v.02'), Synset('treat.v.01

In [39]:
total = 0
total2 = 0
for senti_word_text in sentiment_score_list:
    senti_synset = swn.senti_synset(senti_word_text.name())
    print("| {0}的正向情緒分數:{1} | 負向情緒分數:{2} | 客觀情緒分數:{3} |".format(senti_synset.synset.name(),
                                    senti_synset.pos_score(),
                                   senti_synset.neg_score(),
                                   senti_synset.obj_score()))
    total2 = senti_synset.neg_score()+total2
    total = senti_synset.pos_score()+total

| be.v.01的正向情緒分數:0.25 | 負向情緒分數:0.125 | 客觀情緒分數:0.625 |
| be.v.02的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| be.v.03的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| exist.v.01的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| be.v.05的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| equal.v.01的正向情緒分數:0.125 | 負向情緒分數:0.125 | 客觀情緒分數:0.75 |
| constitute.v.01的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| be.v.08的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| embody.v.02的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| be.v.10的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| be.v.11的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| be.v.12的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| cost.v.01的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| are.n.01的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| cardinal.s.01的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| central.a.02的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| central.n.01的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| theme.v.01的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| subject.n.01的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| theme.n.02的正

In [40]:
print("正向情緒分數加總",total)
print("負向情緒分數加總",total2)
if(total>total2):
    print("正面評論")
elif(total<total2):
    print("負面評論")

正向情緒分數加總 2.0
負向情緒分數加總 2.875
負面評論


# 只取adj，adv

In [41]:
sentiment_score_list = []
for senti_word_text in try_sentence:
    synsets_adj = wn.synsets(senti_word_text,pos=wn.ADJ)
    synsets_adv = wn.synsets(senti_word_text,pos=wn.ADV)
    #synsets_verb = wn.synsets(senti_word_text,pos=wn.VERB)
    #synsets_noun = wn.synsets(senti_word_text,pos=wn.NOUN)
    temp = synsets_adj+synsets_adv
    sentiment_score_list = sentiment_score_list + temp
print(sentiment_score_list) 

[Synset('cardinal.s.01'), Synset('central.a.02'), Synset('handled.a.01'), Synset('stereotypically.r.01'), Synset('no.a.01'), Synset('no.r.01'), Synset('no.r.02'), Synset('no.r.03')]


In [42]:
total = 0
total2 = 0
for senti_word_text in sentiment_score_list:
    senti_synset = swn.senti_synset(senti_word_text.name())
    print("| {0}的正向情緒分數:{1} | 負向情緒分數:{2} | 客觀情緒分數:{3} |".format(senti_synset.synset.name(),
                                    senti_synset.pos_score(),
                                   senti_synset.neg_score(),
                                   senti_synset.obj_score()))
    total2 = senti_synset.neg_score()+total2
    total = senti_synset.pos_score()+total

| cardinal.s.01的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| central.a.02的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| handled.a.01的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| stereotypically.r.01的正向情緒分數:0.125 | 負向情緒分數:0.0 | 客觀情緒分數:0.875 |
| no.a.01的正向情緒分數:0.0 | 負向情緒分數:0.0 | 客觀情緒分數:1.0 |
| no.r.01的正向情緒分數:0.375 | 負向情緒分數:0.25 | 客觀情緒分數:0.375 |
| no.r.02的正向情緒分數:0.0 | 負向情緒分數:0.5 | 客觀情緒分數:0.5 |
| no.r.03的正向情緒分數:0.0 | 負向情緒分數:0.625 | 客觀情緒分數:0.375 |


In [43]:
print("正向情緒分數加總",total)
print("負向情緒分數加總",total2)
if(total>total2):
    print("正面評論")
elif(total<total2):
    print("負面評論")

正向情緒分數加總 0.5
負向情緒分數加總 1.375
負面評論


In [57]:
for senti_word_text in sentiment_score_list:
    senti_synset = swn.senti_synset(senti_word_text.name())
    print(senti_synset.synset)

Synset('cardinal.s.01')
Synset('central.a.02')
Synset('handled.a.01')
Synset('stereotypically.r.01')
Synset('no.a.01')
Synset('no.r.01')
Synset('no.r.02')
Synset('no.r.03')


In [59]:
#print (','.join(','.join(str(v) for v in sentiment_score_list))

In [58]:
#print(','.join(map(str, sentiment_score_list)))

# Varder套件

In [35]:
# -*- coding: utf-8 -*-
from nltk import tokenize
from __future__ import print_function
from nltk.sentiment.vader import SentimentIntensityAnalyzer


sentences =["These are the central themes of the film and they are handled ineptly, stereotypically and with no depth of imagination."]
#sentences =["sweet"]

sid = SentimentIntensityAnalyzer()

for sentence in sentences:
    print("\n",sentence)
    ss = sid.polarity_scores(sentence)
    for k in sorted(ss):
            print("{0}:{1}".format(k, ss[k]),end=" ")
print()


 These are the central themes of the film and they are handled ineptly, stereotypically and with no depth of imagination.
compound:-0.296 neg:0.104 neu:0.896 pos:0.0 


In [44]:
string = "happy"

synset_list = wn.synsets(string)


In [45]:
synset_list

[Synset('happy.a.01'),
 Synset('felicitous.s.02'),
 Synset('glad.s.02'),
 Synset('happy.s.04')]

In [47]:
synset_list[0].name()

'happy.a.01'

In [27]:
senti_score = swn.senti_synset(synset_list[0].name())
print(senti_score)

<happy.a.01: PosScore=0.875 NegScore=0.0>


In [28]:
senti_score.neg_score()

0.0

In [29]:
senti_score.pos_score()

0.875

In [30]:
senti_score.obj_score()

0.125

In [37]:
senti_score.synset.name()

'happy.a.01'

In [49]:
for ii in synset_list:
    print("{0}, Positive:{1}, Negative:{2}".format(swn.senti_synset(ii.name()).synset.name(),
                                                   swn.senti_synset(ii.name()).pos_score(),
                                                   swn.senti_synset(ii.name()).neg_score()))

happy.a.01, Positive:0.875, Negative:0.0
felicitous.s.02, Positive:0.75, Negative:0.0
glad.s.02, Positive:0.5, Negative:0.0
happy.s.04, Positive:0.125, Negative:0.0
